# Robuste Programmierung mit Rust
## Memory Safe, Type Safe, Performant - Pick Three
### Dorian A. Prill | FH Salzburg | Department IT
  
<img align="left" src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Rust_programming_language_black_logo.svg/1024px-Rust_programming_language_black_logo.svg.png" width="350" height="350" />
<img align="left" width="200" height="200" src="https://rustacean.net/assets/rustacean-orig-noshadow.svg"> 


# In diesem Workshop beantworten wir
- Warum sollte ich mit mit neuen Werkzeugen befassen?

- Was bedeutet Memory/Type safety?

- Warum ist das ein Problem?

- Welche Lösungen gibt es allgemein?

- Warum Rust statt XYZ?

- Wann nicht Rust?

- Ausgewählte Beispiele vs C/C++ (Appetizer)

# Warum sollte ich mich mit neuen Werkzeugen befassen?

- Generell ermöglichen bessere Werkzeuge überhaupt erst die Verbesserung von Prozess und Produkt  
- Mit Speer, Keil, Feuer (nicht notwendigerweise in dieser Reihenfolge) fing unsere Zivilisation an  
- Werkzeuge funktionieren am besten in einer Nische, müssen in der Praxis jedoch ein Spektrum abdecken ("Two-Language-Problem") 
- Fertigung: Weniger Verschnitt, kürzere Produktionszeiten, oder ganz neue Technologien (Additive Manufacturing)  
- Software im Großen und Ganzen: Produktiver (Mehr Abstraktion) bei gleicher oder besserer Ausführgeschw. Komerziell weniger wichtig war bisher die Sicherheit (Safety -> Security). 

## Reminder: Was sind Memory und Type Safety?
- **Memory Safety** is the state of being protected from various software bugs and security vulnerabilities when dealing with memory access, such as buffer overflows and dangling pointers[\[1\]](https://en.wikipedia.org/wiki/Memory_safety#cite_note-Adve-1). (Kein deutschsprachiger Artikel vorhanden). Memory Safety kann weitreichende Folgen haben, man kann unterscheiden in 
    - Räumliche Abh. (Out-Of-Bounds Fehler) 
    - Zeitliche Abh. (Use-After-Free)
    - Zugriffsabh.   (Data-Races)  
    
 Um dies zu gewährleisten, benötigen wir auch 


- **Type Safety** bezeichnet den Zustand (einer Programmausführung), bei dem die Datentypen gemäß ihren Definitionen in der benutzten Programmiersprache verwendet werden und keine Typverletzungen auftreten.[\[2\]](https://de.wikipedia.org/wiki/Typsicherheit). Type Safety verbietet in der Regel die Null Referenz und verhindert somit einige Segfaults, Use-After-Free, Double-Free und weitere Fehler und mindert so die Angriffsoberfläche.

Tony Hoare selbst nennt seine Erfindung der Null Referenz seinen ["billion-dollar mistake"](https://www.infoq.com/presentations/Null-References-The-Billion-Dollar-Mistake-Tony-Hoare/) (Ironischerweise teilt Rusts Initiator Graydon Hoare ebenfalls den Namen, es besteht jedoch keine Verwandtschaft)

# Memory Safety offiziell 
## Teil der US-amerikanischen National Security Agency (NSA) "Best Practices" und White House Cybersecurity-Policy

> “We have to consistently use memory safe languages and other protections when developing software to eliminate these weaknesses from malicious cyber actors.”

[NSA Press Release Nov 2022](https://www.nsa.gov/Press-Room/News-Highlights/Article/Article/3215760/nsa-releases-guidance-on-how-to-protect-against-software-memory-safety-issues/)


> Cybersecurity solutions should be informed by engineering best practices, and technology
manufacturers building software can tackle this issue by consistently using secure building blocks;
specifically, adopting memory safe programming languages. There is strong evidence that now is
the time to make these changes. First, technical solutions already exist; there are dozens of memory
safe programming languages that can – and should – be used.

[White House Publication Feb 2024](https://www.whitehouse.gov/wp-content/uploads/2024/02/Final-ONCD-Technical-Report.pdf)

# Beispiel: Dies ist valider C++ Code
```cpp  
#include <vector>
#include <iostream>

int main() {
    std::vector<int> vec = {1, 2, 3, 4, 5};

    int& ref = vec[0];

    vec.push_back(6);

    std::cout << "Ref: " << ref << std::endl;

    return 0;
}
```
Wo ist hier das Problem?

# Beispiel: Dies ist *invalider* Rust Code

```rust
fn main() {
    let mut vec = vec![1, 2, 3, 4, 5];

    // Immutably borrow an element from the vector
    let ref_element = &vec[0];

    // Try to modify the vector while the immutable borrow exists
    vec.push(6); // Compile-time error!

    println!("Ref: {}", ref_element);
}
```
Kompiliert nicht, Fehlermeldung:

```
error[E0502]: cannot borrow `vec` as mutable because it is also borrowed as immutable
  --> src/main.rs:8:5
   |
5  |     let ref_element = &vec[0];
   |                        --- immutable borrow occurs here
...
8  |     vec.push(6); // Compile-time error!
   |     ^^^^^^^^^^^ mutable borrow occurs here
9  |
10 |     println!("Ref: {}", ref_element);
   |                         ----------- immutable borrow later used here

For more information about this error, try `rustc --explain E0502`.
```

# Beispiel: Initialisierungszwang

```rust
fn main() {
    let x: i32;
    println!("{}", x);
}
```

```
error[E0381]: borrow of possibly-uninitialized variable: `x`
 --> src/main.rs:3:20
  |
3 |     println!("{}", x);
  |                    ^ borrow of possibly-uninitialized variable
```

```cpp
#include <iostream>
int main() {
    int x;
    std::cout << x << std::endl;
}
```
Funktioniert, gibt jedoch keinen sinnvollen Wert aus.

## Memory Safety bei Microsoft
![microsoft_memory_cve](./resource/images/microsoft_study_cve_memsafety.png)
Quelle: Microsoft Security Response Center [Blog 1](https://msrc-blog.microsoft.com/2019/07/18/we-need-a-safer-systems-programming-language/) und [Blog 2](https://msrc-blog.microsoft.com/2019/07/16/a-proactive-approach-to-more-secure-code/) und 
[Ausführliche Präsentation](https://github.com/microsoft/MSRC-Security-Research/tree/master/presentations/2019_02_BlueHatIL) (siehe Slide 10).   
Grafik wurde nicht erneuert für 2019-2020, Aktuelle Fallzahlen legen nahe, dass der Einsatz von Rust diese Fehlerquelle reduziert/mitigiert (Siehe Beispiel Android später)

## Memory Safety in Chromium
![chromium_memory_cve](https://www.chromium.org/Home/chromium-security/memory-safety/piechart.png) 

"These bugs are spread evenly across our codebase, and a high proportion of our non-security stability bugs share the same types of root cause. As well as risking our users’ security, these bugs have real costs in how we fix and ship Chrome."
Quelle: [The Chromium Project](https://www.chromium.org/Home/chromium-security/memory-safety).

Vorschläge des Projekts: Safety Bibliotheken/Dialekte in C++ und sichere Sprachen

# Mehr Memory (Un-)Safety
- Firefox CSS Subsystem 73.9 % der Sicherheitslücken [Quelle](https://hacks.mozilla.org/2019/02/rewriting-a-browser-component-in-rust/)
- iOS: 66.3 %  und  macOS: 71.5 % aller Sicherheitslücken [Quelle](https://langui.sh/2019/07/23/apple-memory-safety/)
- Eine erweiterte Zusammenfassung ist [bei Alex Gaynor (unaffiliated) zu finden](https://alexgaynor.net/2020/may/27/science-on-memory-unsafety-and-security/)

Nachdem wir nun empirische Beweise gesammelt haben, stellen wir eben diese Beweislage gegenüber gängiger Meinung/Praxis

## Gängige Hypothese
> "Gute C/C++ Programmierer, wissen wie man Fehler vermeidet."

## Erklärung 1
~70% *aller* C/C++ Programme werden folglich nicht von C/C++ Programmierern geschrieben.

## Erklärung 2
~70% *aller* C/C++ Programmierer sind folglich keine "guten" Programmierer. 

(shamelessly stolen from Ryan Levick)

# Was bedeutet Memory Safety für mein Unternehmen?
- CVE steht für Common Vulnerabilities and Exposures, also ausnutzbare Sicherheitslücken!
- Personenbezogene Daten als (kostspielige) Verantwortung: GDPR (General Data Protection Regulation)
- Kritische Steuerungsanwendungen (Energie, Autonome Fahrzeuge, IoT Geräte)
- Lange Lebenszyklen profitieren besonders von robuster Software (Wartbarkeit)

## Nutzung in bekannten Unternehmen
- Mozilla (Firefox/Servo, WebRender)
- Microsoft (Azure/IoT Products angeblich noch weitere *Ryan Levick)
- Dropbox ([Sync Engine](https://www.wired.com/2016/03/epic-story-dropboxs-exodus-amazon-cloud-empire/)) 
- NPM Server (JavaScript Paket-Manager) 
- Facebook (Libra Blockchain)
- Amazon AWS Nitro ([Jobangebot](https://aws.amazon.com/blogs/opensource/why-aws-loves-rust-and-how-wed-like-to-help/))
- Google (Android, others)

Für alle diese Projekte ist Performanz und Sicherheit wichtig. Zudem sind die Großunternehmen nicht nur auf den Wagen aufgesprungen, sondern haben sogar Pferde davorgespannt, indem sie die Rust Foundation fördern.

# Aktuelles Beispiel: Google Android
Aus dem [Google Security Blog](https://security.googleblog.com/2022/12/memory-safe-languages-in-android-13.html):  
>   Android 13 is the first Android release where a majority of new code added to the release is in a memory safe language  
    ...  
    2022 is the first year where memory safety vulnerabilities do not represent a majority of Android’s vulnerabilities  
    ...  
    To date, there have been zero memory safety vulnerabilities discovered in Android’s Rust code.
    
 Memory-Safe Sprachen in Android sind Java/Kotlin/Rust, mit dem groessten Anteil in Java. Rust wird besonders für low-level Komponenten verwendet, wie Kernel/-Module.


> In Android 13, about 21% of all new native code (C/C++/Rust) is in Rust. There are approximately 1.5 million total lines of Rustcode in AOSP... To date, there have been zero memory safety vulnerabilities discovered in Android’s Rust code. ... It demonstrates that Rust is fulfilling its intended purpose of preventing Android’s most common source of vulnerabilities. ... Historical vulnerability density is greater than 1/kLOC (1 vulnerability per thousand lines of code) ... Based on this historical vulnerability density, it’s likely that using Rust has already prevented hundreds of vulnerabilities from reaching production.

Google's Strategie ist

> ... our goal is not to convert existing C/C++ to Rust, but rather to shift development of new code to memory safe languages over time. 

Ähnliches passiert auch bei Microsoft (Win11 GraphicsDeviceInterface und SysCall lt. R. Levick und D. Weston)

# Warum Rust und nicht ...? I
|                    | Rust                | C++                    | C\#                 | Swift      | Golang     | Ada                  |
| ------------------ |:-------------------:|:---------------------:|:-------------------:|:----------:|:----------:|:--------------------:|
| Build Tools        | ja (cargo)          | 3rd party              | ja (MSBuild)        | ja         | ja         | ja (GNAT, gprbuild)  |
| Paketmanager       | ja, (cargo)         | 3rd party              | ja (NuGet)          | ja         | ja         | nein (3rd party Lösungen vorhanden) |
| Memory Management  | semi-manuell        | manuell  (semi durch libs)              | auto (GC)           | semi-auto (rc+manuell) | auto (GC)   | manuell/semi-manuell |
| Spatial Safety     | ja                  | nein/begrenzt (Libs, ext. Werkzg., Dialekte) | ja          | ja         | ja         | ja                    |
| Temporal Safety    | ja                  | nein/begrenzt,(Libs, ext. Werkzg., Dialekte) | ja          | ja         | ja         | ja                    |
| Zugriffsicherheit  | ja                  | nein                   | nein ([CHESS](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/chess-chesspldi2009tutorial.pdf)) | nein  | nein       | ja                    |
| Null-Safety        | ja                  | nein                   | ja, seit C\# 8      | ja         | nein       | ja (mittels Ada 2012 Aspects) |
 


Für Sprachen, bei denen die Safety-Features nicht bestehen, können Programme zur statischen Analyse helfen. Diese können allerdings nicht alle Fehler finden. Beispiele: [List of Static Analyzers](https://github.com/analysis-tools-dev/static-analysis#c)
Dynamische Analyse bez. Performance und Input Handling müssen trotzdem noch zur Laufzeit durchgeführt werden (Fuzzer, Valgrind, Flamegraph etc.)

# Warum Rust und nicht ...? II
Rust ist also ideal geeignet für Anwendungen, in denen traditionell Sprachen wie C/C++/Ada/Fortran verwendet werden, wildert durch die moderne Ergonomie aber auch in den "unteren" Bereichen der Managed Sprachen. Gleichzeitig hebt es sich durch die nicht benötigte Garbage Collection (=bessere Performanz) ab.

Bisher gab es eigentlich nur drei Zertifizierungen für diesen Bereich durch:
- [MISRA C](https://misra.org.uk/) (Autoindustrie, Raumfahrt)
- [AUTOSAR C++](https://www.autosar.org/standards/classic-platform) (Autoindustrie)
- [Ada/SPARK](https://www.adacore.com/about-spark) (Raumfahrt- und Verteidigungsindustrie)

Rust hat mittlerweile auch eine zertifizierte Toolchain für sicherheitskritische Anwendungen, welche in diese Bereichen vordringen will:
- aktuell: ISO 26262 (ASIL D) und IEC 61508 (SIL 4)
- geplant: DO-178C, ISO 21434 und IEC 62278

# Wann nicht Rust?

- Wenn Ihre Anforderungen durch Managed-Sprachen ausreichend abgedeckt sind (C\#, Java) -> Einheitliches Ökosystem, geringere Einarbeitungskosten
- Wenn Sie auf bestehenden Code aufbauen, der nicht in Rust geschrieben ist und realistisch nicht umgeschrieben werden kann (z.B. COBOL + Zertifizierungen)
- Wenn Ihr Produkt auf eine stabile ABI (Application Binary Interface) angewiesen ist (die z.B. C/Ada bietet)
- Ihre Arbeit hauptsächlich bestehende Komponenten integriert bzw. auf Frameworks aufbaut (Data Science, Web Development, Game Development)
- Wenn zeitliche und finanzielle Ressourcen begrenzt sind und die Einarbeitung in Rust zu lange dauern würde bzw. Entwickler zu teuer sind

## Kleingedrucktes 
- Data Races in Rust können ohne unsafe{..} weder räumliche noch zeitliche Sicherheit verletzen!
- Rust threads können trotzdem im Deadlock enden. Die Garantie gilt nur für shared Memory Safety.
- Rusts Sicherheit bezahlt man mit Implementierungs- und Compile-Zeit (spart aber an Debugging).  
- Slices in Rust kennen ihre eigene Größe und verursachen Panics statt ausnutzbare Out-Of-Bounds Fehler.  
- Rust hat (noch) keine Spezifikation, ist also für bestimmte Bereiche noch nicht geeignet (nicht wie Ada/SPARK)
- NEU: Rust hat mit Ferrocene jetzt einen zertifizierten Compiler
- Rust ist so schnell wie C/C++, dank sog. Zero-Cost-Abstractions

### In der Regel gilt: Kompiliert das Programm, führt es zu _keinem unkontrollierten(!)_ Programmabsturz

# Empfohlene Installation

- [rustup](https://rustup.rs/) (Rust Toolchain Manager)
- [Visual Studio Code](https://code.visualstudio.com/) (IDEtor :)
- [VSCode Plugin rust-analyzer](https://marketplace.visualstudio.com/items?itemName=matklad.rust-analyzer) (VSCode Extension)
- [VSCode Plugin crates](https://marketplace.visualstudio.com/items?itemName=serayuzgur.crates) (VSCode Extension) (Library Version ("crate" in Rust) Management)
- [VSCode Plugin CodeLLDB](https://marketplace.visualstudio.com/items?itemName=vadimcn.vscode-lldb) (VSCode Extension) (LLVM Debugger)
- [VSCode Plugin Even Better TOML](https://marketplace.visualstudio.com/items?itemName=tamasfe.even-better-toml) (VSCode Extension) (TOML Syntax Highlighting)

Jetzt wollen wir ein einfaches Projekt anlegen und kompilieren.

# Konzept Ownership
1. Zu jedem Wert in Rust gibt es Variable, die ihn Besitzt
2. Es kann immer nur einen Besitzer zur Zeit geben
3. Wenn der Gültigkeitsbereich des Besitzers verlassen wird, wird der Wert fallengelassen

## Gültigkeitsbereiche von Variablen
Value Scope: Referenz auf ein String Literal '&str' auf dem Stack

In [ ]:
{                            // s gilt hier nicht, es wurde noch nicht deklariert
    let s = "bonjour.";      // ab diesem Punkt gilt die Variable s
    s.to_uppercase();        // string Operation erzeugt einen neuen 'String', da string literale immutable sind
    println!("{}",s) 
}                            // Gültigkeitsbereich von s vorbei

Der Heap String Typ kann dagegen verändert werden:

In [ ]:
{
let mut s = String::from("Moin");         // Speicher wird vom Allokator "ausgeliehen"

s.push_str(", Servus, Grüzi und Hallo!"); // push_str() fügt ein Literal and den String an

println!("{}", s);
}                                         // Gültigkeitsbereich von s endet, Speicher wird autom. zurückgegeben (drop())

Bis hierhin verhält sich Rust also wie modernes C++ mit RAII Pattern: Resource Acquisition Is Initialization

## Ownership: Zugehörigkeit verschieben (Move Semantics)


In [ ]:
let s1 = String::from("hello");
let s2 = s1;
println!("{}, world!", s1);

<img align="left" width="450" height="600" src="https://doc.rust-lang.org/book/img/trpl04-04.svg">
Um eine doppelte Referenz zu vermeiden (und folglich ein Double-Free), wird s1 ungültig, sobald die Referenz verschoben wird ("Move"). Das ist der wesentliche Unterschied zur "Shallow-Copy" in anderen Sprachen (Dort wird [Pointer, Länge, Kapazität] auch kopiert, aber beide Referenzen werden behalten)

# Konzept Borrowing
- Rust erlaubt es, eine Referenz auf einen Wert zu erstellen, ohne den Besitz zu übernehmen
- Es gibt zwei Arten von Referenzen: mutable und immutable
- Es kann beliebig viele immutable Referenzen geben, aber nur eine mutable Referenz
- Referenzen müssen immer gültig sein, solange sie existieren
- Referenzen können nicht über den Gültigkeitsbereich des Besitzers hinausgehen
- Referenzen besitzen nie den Wert, auf den sie zeigen, die Ownership bleibt beim Besitzer
- Borrowing ist eine der wichtigsten Eigenschaften von Rust, die es erlaubt, Memory Safety zu garantieren

## Offizielle Resourcen  
 - [Offizielle Seite von Rust-Lang](https://www.rust-lang.org/)
 - [Offizielle Rust Learning Resourcen](https://www.rust-lang.org/learn) Enthält ganze Bücher für Sprache/Toolchain/WASM/Embedded und interaktive Kurse
 
## Weitere Resourcen (Dev Blogs, Awesome-Lists)
 - [Why Rust is the Future of Game Development](https://thefuntastic.com/blog/why-rust-is-the-future-game-dev) Klickköder Titel aber sehr objektive und weitreichende Informationen
 - [rust dev withoutboats' blog](https://without.boats/blog/) Inhalte für technisch interessierte aus Rust dev sicht
 - [Awesome Rust Liste](https://github.com/rust-unofficial/awesome-rust) Überblick über die reifsten Bibliotheken je Kategorie
 - [AreWeWebYet](https://www.arewewebyet.org/) Sind wir schon bereit für ... web?
 - [AreWeGUIYet](https://www.areweguiyet.com/) Sind wir schon bereit für ... GUI?
 - [AreWeGameYet](https://arewegameyet.rs/) Sind wir schon bereit für ... games?
 - [AreWeIDEYet](https://areweideyet.com/) Sind wir schon bereit für ... IDEs?
 
 